### Установка библиотеки и подключение к диску.


In [ ]:
import os
import numpy as np

In [ ]:
!python -m pip install -U giotto-tda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.4/526.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.4 MB/s eta 0:00:00


In [ ]:
!pip install gudhi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.9 MB/s eta 0:00:00


In [ ]:
!pip install pot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 9.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Загружаем данные

Функция от названия файла, возвращает массив (245, 600) - данные RS_0 и RS_FE

In [ ]:
from numpy import load

def file_to_array(filename):
    archive = load(filename)
    files = archive.files
    array = archive['data']
    shape = archive['shape']
    indptr = archive['indptr']

    data = np.empty((1, shape[1]))

    for i in range(shape[0]):
      data = np.append(data, [array[indptr[i]:indptr[i + 1]]], axis=0)

    return data[1:][:, :300], data[1:][:, 300:]


rs_0, rs_fe = file_to_array('/content/drive/MyDrive/hse/kursach1/human fmri/human fmri exp rs_0 vs. fe merged p1.npz')

In [ ]:
rs_0.shape

(245, 300)

### Получение матрицы ковариаций и персистентной диаграммы

Считаем матрицу ковариаций и получаем граф.

In [ ]:
import pandas as pd
import numpy as np

def corr_matrix(data, param):
    '''
    Computing covariance matrix of data cloud, inf if cov < param
    '''
    f = lambda x: abs(x) if abs(x) >= param else 0
    vfunc = np.vectorize(f)

    corr_array = np.corrcoef(data)

    corr_array = vfunc(corr_array)
    np.fill_diagonal(corr_array, 0)

    return [1 - corr_array]

corr_array = corr_matrix(rs_0, 0.6)
corr_array[0].shape

(245, 245)

Персистентная диаграмма по графу для пациента p1


In [ ]:
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram

def pers_diagram(data, n_homologies):
    homology_dimensions = [_ for _ in range(n_homologies)]

    persistence = VietorisRipsPersistence(
        homology_dimensions=homology_dimensions,
        n_jobs=6,
        metric="precomputed"
    )
    diagram = persistence.fit_transform(data)
    # diagram = diagram[~np.any(diagram == np.inf, axis=2)][np.newaxis, :, :]
    return diagram
plot_diagram(pers_diagram(corr_array, 3)[0])

### Делаем диаграммы устойчивости для каждого пациента rs_0 и rs_fe  и загружаем на диск

In [ ]:
param = 0.6
def get_diagrams_array(param: int, n_hom: int):
    X = []
    for i in range(1, 24):
        filename = f'/content/drive/MyDrive/hse/kursach1/human fmri/human fmri exp rs_0 vs. fe merged p{i}.npz'
        rs_0, rs_fe = file_to_array(filename)
        corr_rs_0 = corr_matrix(rs_0, param)
        corr_rs_fe = corr_matrix(rs_fe, param)

        rs_0_persistent = pers_diagram(corr_rs_0, n_hom)[:, :, :2]
        rs_fe_persistent = pers_diagram(corr_rs_fe, n_hom)[:, :, :2]

        X.append(rs_0_persistent)
        X.append(rs_fe_persistent)
        # print(f"level {i} completed!")
    for i in range(0, 46):
        X[i] = np.delete(X[i], np.where(X[i] == [0., 0.]), axis=1)
    return X

# X = get_diagrams_array(param, 3)


Сохраняю в виде .npz на диске

In [ ]:
# для [1 - corr_array]
for i in range(0, 46):
  X[i] = np.delete(X[i], np.where(X[i] == [0., 0.]), axis=1)
  # X[i] = X[i][:, 1:, :]
  print(X[i].shape)
X[i].shape

(1, 62, 2)
(1, 41, 2)
(1, 116, 2)
(1, 100, 2)
(1, 77, 2)
(1, 98, 2)
(1, 74, 2)
(1, 105, 2)
(1, 131, 2)
(1, 90, 2)
(1, 74, 2)
(1, 46, 2)
(1, 149, 2)
(1, 125, 2)
(1, 131, 2)
(1, 136, 2)
(1, 104, 2)
(1, 113, 2)
(1, 58, 2)
(1, 86, 2)
(1, 66, 2)
(1, 34, 2)
(1, 92, 2)
(1, 110, 2)
(1, 67, 2)
(1, 112, 2)
(1, 103, 2)
(1, 52, 2)
(1, 93, 2)
(1, 52, 2)
(1, 142, 2)
(1, 105, 2)
(1, 96, 2)
(1, 65, 2)
(1, 94, 2)
(1, 45, 2)
(1, 63, 2)
(1, 16, 2)
(1, 112, 2)
(1, 117, 2)
(1, 111, 2)
(1, 100, 2)
(1, 138, 2)
(1, 95, 2)
(1, 65, 2)
(1, 59, 2)


(1, 59, 2)

In [ ]:
import numpy as np
for i in range(0, 46, 2):
  np.savez(f'/content/drive/MyDrive/hse/kursach1/results/pers diagram rs_0 p{i // 2 + 1}.npz', X[i])
  np.savez(f'/content/drive/MyDrive/hse/kursach1/results/pers diagram rs_fe p{i // 2 + 1}.npz', X[i + 1])
  print(X[i].shape)

(1, 62, 2)
(1, 116, 2)
(1, 77, 2)
(1, 74, 2)
(1, 131, 2)
(1, 74, 2)
(1, 149, 2)
(1, 131, 2)
(1, 104, 2)
(1, 58, 2)
(1, 66, 2)
(1, 92, 2)
(1, 67, 2)
(1, 103, 2)
(1, 93, 2)
(1, 142, 2)
(1, 96, 2)
(1, 94, 2)
(1, 63, 2)
(1, 112, 2)
(1, 111, 2)
(1, 138, 2)
(1, 65, 2)


### Загружаем диаграммы устойчивости с диска

In [ ]:
X = []

def load_data(filename):
  archive = load(filename)
  files = archive.files
  array = archive['arr_0']
  return array

for i in range(0, 46, 2):
  rs_0 = load_data(f'/content/drive/MyDrive/hse/kursach1/results/pers diagram rs_0 p{i // 2 + 1}.npz')
  rs_fe = load_data(f'/content/drive/MyDrive/hse/kursach1/results/pers diagram rs_fe p{i // 2 + 1}.npz')
  X.append(rs_0)
  X.append(rs_fe)

len(X)

46

In [ ]:
for i in range(46):
  print(X[i].shape)

(1, 38, 2)
(1, 30, 2)
(1, 90, 2)
(1, 70, 2)
(1, 59, 2)
(1, 62, 2)
(1, 46, 2)
(1, 73, 2)
(1, 95, 2)
(1, 66, 2)
(1, 59, 2)
(1, 23, 2)
(1, 108, 2)
(1, 95, 2)
(1, 99, 2)
(1, 98, 2)
(1, 78, 2)
(1, 85, 2)
(1, 42, 2)
(1, 50, 2)
(1, 47, 2)
(1, 20, 2)
(1, 62, 2)
(1, 84, 2)
(1, 57, 2)
(1, 81, 2)
(1, 75, 2)
(1, 28, 2)
(1, 70, 2)
(1, 45, 2)
(1, 106, 2)
(1, 88, 2)
(1, 71, 2)
(1, 46, 2)
(1, 73, 2)
(1, 30, 2)
(1, 41, 2)
(1, 15, 2)
(1, 81, 2)
(1, 96, 2)
(1, 87, 2)
(1, 63, 2)
(1, 95, 2)
(1, 70, 2)
(1, 44, 2)
(1, 37, 2)


### K-means


In [ ]:
%%time
import numpy as np
from sklearn.datasets import make_blobs
from gudhi.wasserstein import wasserstein_distance
from gudhi.wasserstein.barycenter import lagrangian_barycenter
import time

class KMeans:
    def __init__(self, n_clusters=2, max_iters=10):
        self.n_clusters = n_clusters
        self.max_iters = max_iters

    def wasserstein(self, diagram):
        # Compute Wasswrstein distance from first and second centroid to diagram
        return [wasserstein_distance(diagram[0], self.centroids[0][0]), wasserstein_distance(diagram[0], self.centroids[1][0])]

    def match_centroids(self, new_centroids):
        return (new_centroids[0].shape == self.centroids[0].shape or new_centroids[0].shape == self.centroids[1].shape) \
        and (new_centroids[1].shape == self.centroids[1].shape or new_centroids[1].shape == self.centroids[0].shape) \
        and np.all([new_centroids[0], self.centroids[0]]) and np.all([new_centroids[1], self.centroids[1]])

    def fit(self, X):
        answers = np.zeros(len(X))
        answers[::2] = 1
        # Initialize centroids randomly
        c1, c2 = np.random.choice(len(X), self.n_clusters, replace=False)
        self.centroids = [X[c1], X[c2]]

        for _ in range(self.max_iters):
            # print(f'Starting iteration {_ + 1}')
            # Assign each data point to the nearest centroid
            labels = self.assign_labels(X)

            # Update centroids
            new_centroids = self.update_centroids(X, labels)

            matches = max(np.sum(answers == labels), np.sum(1 - answers == labels))

            # print(f'{(matches / len(X)) * 100}% match after updating centroids\n')

            if self.match_centroids(new_centroids):
              return

            self.centroids = new_centroids

    def assign_labels(self, X):
        # Compute wasserstein distances from each data point to centroids
        distances = np.array(list(map(self.wasserstein, X)))
        # Assign labels based on the nearest centroid
        return np.argmin(distances, axis=1)

    def update_centroids(self, X, labels):
        # Regroup diagrams
        groups = [[] for _ in range(self.n_clusters)]
        for i in range(len(labels)):
            groups[labels[i]].append(X[i][0])

        # for i in range(self.n_clusters):
        #     print(f'Size of {i} cluster is {len(groups[i])}')
        # # Compute new barycenters
        new_centroids = []
        for i in range(self.n_clusters):
            # print('Computing barycenter...')
            centroid = lagrangian_barycenter(pdiagset=groups[i])[np.newaxis, :, :]
            new_centroids.append(centroid)

        return new_centroids

def cluster(X: np.array, n: int):
    start = time.time()

    kmeans = KMeans(2, 10)
    kmeans.fit(X)
    labels = kmeans.assign_labels(X)

    end = time.time() - start

    return labels
# print("Centroids:", kmeans.centroids)

CPU times: user 81 µs, sys: 0 ns, total: 81 µs
Wall time: 85.4 µs


In [ ]:
import pandas as pd

parameters = [0.6, 0.65, 0.7, 0.75, 0.8]
number_of_homologies = [2, 3]

df = pd.DataFrame({'n_homologies': [], 'corr_param': [], 'labels': [], 'percentage': []})

def get_percentage(labels: np.array) -> int:
    answers = np.zeros(len(labels))
    answers[::2] = 1
    matches = max(np.sum(answers == labels), np.sum(1 - answers == labels))

    return (matches / len(labels)) * 100


for param in parameters:
    for n_hom in number_of_homologies:
        X = get_diagrams_array(param, n_hom)
        labels = cluster(X, 10)
        labels_str = ' '.join([str(int(label)) for label in labels])
        new_row = {'n_homologies': n_hom, 'corr_param': param, 'labels': labels_str, 'percentage': get_percentage(labels)}
        df = df.append(new_row, ignore_index=True)
        print(f'homology {n_hom} with parameter {param} done')



<ipython-input-27-c6320d82251c>:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



homology 2 with parameter 0.6 done


<ipython-input-27-c6320d82251c>:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



homology 3 with parameter 0.6 done


<ipython-input-27-c6320d82251c>:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



...

<ipython-input-27-c6320d82251c>:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



homology 3 with parameter 0.75 done


<ipython-input-27-c6320d82251c>:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



homology 2 with parameter 0.8 done
homology 3 with parameter 0.8 done


<ipython-input-27-c6320d82251c>:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [ ]:
df

,n_homologies,corr_param,labels,percentage
0,2.0,0.60,1 0 0 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1 1 ...,56.521739
1,3.0,0.60,1 0 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1 1 ...,52.173913
2,2.0,0.65,1 1 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0 ...,58.695652
3,3.0,0.65,0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 1 0 1 ...,63.043478
4,2.0,0.70,0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 1 0 0 ...,65.217391
5,3.0,0.70,0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 1 0 0 ...,63.043478
6,2.0,0.75,1 1 0 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 1 1 1 1 ...,65.217391
7,3.0,0.75,0 0 1 0 0 0 0 0 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 ...,63.043478
8,2.0,0.80,1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 1 1 ...,52.173913
9,3.0,0.80,1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 ...,56.521739


In [ ]:
df.to_csv('/content/drive/MyDrive/hse/kursach1/results/result_table.csv', index=False)

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_avg = silhouette_score(X, kmeans.labels_)
print("The average silhouette_score is :", silhouette_avg)

### Сравнение gudhi и giotto-tda

In [ ]:
import gudhi

def gudhi_pers_diagram(data):

  rips_complex = gudhi.RipsComplex(distance_matrix=data, max_edge_length=1.0)

  simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
  simplex_tree.get_filtration()
  return simplex_tree.persistence()
# len(diag)
# # corr_array[0].shape

In [ ]:
import timeit

time_giotto = timeit.timeit('pers_diagram(corr_array, 2)', number=1, globals=globals())
time_gudhi = timeit.timeit('gudhi_pers_diagram(corr_array[0])', number=1, globals=globals())

print(time_giotto, time_gudhi)

1.057466199000146 2.2219925070000954
